# To do:

 - Figure out how to signal injuries

In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-04


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [5]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [6]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['Season', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

# Main Model

In [7]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [8]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [9]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

96 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
32,CLE,Sam Merrill,SG,DET,23.550919,26.418029,9.5,19.787407
73,CLE,Evan Mobley,PF,DET,31.198790,29.981841,16.5,12.340613
46,CLE,De'Andre Hunter,SF,DET,26.888300,26.372499,13.5,11.228312
31,CLE,Jaylon Tyson,SG,DET,27.216845,27.896196,12.5,14.310860
23,CLE,Donovan Mitchell,SG,DET,34.247589,32.995182,28.5,26.935966
88,CLE,Jarrett Allen,C,DET,27.867365,26.294336,12.5,13.306227
76,CLE,Dean Wade,PF,DET,15.034265,22.933321,5.5,4.715724
11,CLE,Darius Garland,PG,DET,32.487499,31.767260,17.5,17.983263


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
59,MIL,Giannis Antetokounmpo,PF,SAC,28.480701,28.829730,29.5,19.889799
81,MIL,Myles Turner,C,SAC,27.853252,27.893523,10.5,16.727028
9,MIL,Kevin Porter Jr.,PG,SAC,36.455505,35.636871,17.5,14.562097
12,MIL,Ryan Rollins,PG,SAC,31.941858,29.359307,15.5,16.771965
66,MIL,Bobby Portis,PF,SAC,23.641373,26.425074,12.5,13.472248
67,MIL,Kyle Kuzma,PF,SAC,29.801361,29.057417,11.5,11.791162


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
37,DEN,Christian Braun,SG,BRK,27.538017,28.551500,11.5,3.507168
63,DEN,Aaron Gordon,PF,BRK,23.628725,29.773669,12.5,10.482591
7,DEN,Jamal Murray,PG,BRK,36.312431,35.845502,25.5,24.317053
51,DEN,Peyton Watson,SF,BRK,32.882156,29.982040,14.5,13.504377


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
24,SAC,Zach LaVine,SG,MIL,28.486666,29.404528,16.5,23.567083
71,SAC,Keegan Murray,PF,MIL,34.168648,34.698530,15.5,14.087653
14,SAC,Russell Westbrook,PG,MIL,28.383686,29.013702,14.5,13.521154
64,SAC,DeMar DeRozan,PF,MIL,34.672501,30.929404,19.5,19.174313
90,SAC,Maxime Raynaud,C,MIL,28.879982,26.475719,12.5,12.215854
15,SAC,Dennis Schroder,PG,MIL,26.608156,25.601045,11.5,11.510719


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
4,NOP,Jordan Poole,PG,MIA,27.910013,27.161992,16.5,20.284595
61,NOP,Zion Williamson,PF,MIA,30.150599,28.245887,24.5,27.965334
6,NOP,Jeremiah Fears,PG,MIA,27.707008,26.660222,12.5,10.783546
83,NOP,Derik Queen,C,MIA,24.684788,29.352802,12.5,14.184892
47,NOP,Trey Murphy III,SF,MIA,29.734610,34.605384,20.5,19.046286


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
22,OKC,Jalen Williams,SG,PHO,28.511902,28.316282,17.5,14.320476
69,OKC,Chet Holmgren,PF,PHO,29.873045,27.023697,16.5,19.569561
0,OKC,Shai Gilgeous-Alexander,PG,PHO,33.915668,30.848370,31.5,29.364738
36,OKC,Cason Wallace,SG,PHO,26.628942,25.714904,7.5,9.163237
26,OKC,Ajay Mitchell,SG,PHO,26.103598,24.658169,12.5,13.827897
57,OKC,Luguentz Dort,SF,PHO,27.252378,25.669445,6.5,6.818851
30,OKC,Aaron Wiggins,SG,PHO,21.226280,14.174378,9.5,9.222972


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
5,IND,Andrew Nembhard,PG,ORL,33.359459,31.777865,18.5,15.796778
48,IND,Aaron Nesmith,SF,ORL,29.206600,24.039442,13.5,10.986235
43,IND,Ben Sheppard,SG,ORL,20.102419,21.708561,6.5,8.107225
70,IND,Pascal Siakam,PF,ORL,33.805614,31.711855,24.5,24.953512
93,IND,Jay Huff,C,ORL,18.345234,20.914304,7.5,7.204438
10,IND,T.J. McConnell,PG,ORL,17.348101,15.957269,9.5,9.783926


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
40,WAS,Bilal Coulibaly,SG,MIN,28.367390,26.788715,10.5,12.715169
35,WAS,Tre Johnson,SG,MIN,24.575035,21.721966,14.5,15.864750
58,WAS,Justin Champagnie,SF,MIN,24.012918,22.886698,9.5,10.825013
29,WAS,CJ McCollum,SG,MIN,31.834545,31.846917,18.5,17.792765
55,WAS,Khris Middleton,SF,MIN,22.410961,21.819302,9.5,9.033706


../tables/2025/gmday_preds_PTS.csv saved!


In [10]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

75 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
59,MIL,Giannis Antetokounmpo,PF,SAC,28.480701,28.829730,45.5,32.993454
66,MIL,Bobby Portis,PF,SAC,23.641373,26.425074,19.5,23.249355
81,MIL,Myles Turner,C,SAC,27.853252,27.893523,17.5,20.420702
9,MIL,Kevin Porter Jr.,PG,SAC,36.455505,35.636871,29.5,31.691074
12,MIL,Ryan Rollins,PG,SAC,31.941858,29.359307,25.5,26.419693
67,MIL,Kyle Kuzma,PF,SAC,29.801361,29.057417,18.5,18.056938


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
32,CLE,Sam Merrill,SG,DET,23.550919,26.418029,14.5,25.904846
23,CLE,Donovan Mitchell,SG,DET,34.247589,32.995182,38.5,34.339298
88,CLE,Jarrett Allen,C,DET,27.867365,26.294336,22.5,26.658083
73,CLE,Evan Mobley,PF,DET,31.198790,29.981841,29.5,25.635290
46,CLE,De'Andre Hunter,SF,DET,26.888300,26.372499,19.5,16.404209
76,CLE,Dean Wade,PF,DET,15.034265,22.933321,11.5,9.890731
31,CLE,Jaylon Tyson,SG,DET,27.216845,27.896196,20.5,20.812252
11,CLE,Darius Garland,PG,DET,32.487499,31.767260,27.5,27.626432


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
1,LAL,Luka Doncic,PG,MEM,35.530140,32.367308,51.5,43.231621
45,LAL,LeBron James,SF,MEM,33.629990,31.058966,35.5,31.214378
38,LAL,Marcus Smart,SG,MEM,29.761141,25.773956,17.5,14.948503


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
33,MEM,Cedric Coward,SG,LAL,27.743280,27.678212,23.5,31.521452
85,MEM,Jaren Jackson Jr.,C,LAL,33.197258,31.633083,28.5,33.791691
3,MEM,Ja Morant,PG,LAL,30.281227,28.007422,35.5,33.075859
34,MEM,Jaylen Wells,SG,LAL,30.453861,31.675684,18.5,19.582409


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
18,ORL,Tyus Jones,PG,IND,20.738050,23.592847,13.5,10.078809
27,ORL,Desmond Bane,SG,IND,34.898941,35.254938,31.5,28.199833
62,ORL,Paolo Banchero,PF,IND,36.149956,34.159427,39.5,37.134235
8,ORL,Anthony Black,PG,IND,34.568218,34.458659,28.5,30.690632
82,ORL,Wendell Carter Jr.,C,IND,31.493427,31.640240,23.5,21.934959


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
0,OKC,Shai Gilgeous-Alexander,PG,PHO,33.915668,30.848370,42.5,39.196156
22,OKC,Jalen Williams,SG,PHO,28.511902,28.316282,28.5,25.322142
69,OKC,Chet Holmgren,PF,PHO,29.873045,27.023697,27.5,30.631872
26,OKC,Ajay Mitchell,SG,PHO,26.103598,24.658169,18.5,20.979067
36,OKC,Cason Wallace,SG,PHO,26.628942,25.714904,12.5,14.354012


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
53,MIA,Andrew Wiggins,SF,NOP,32.401409,30.546752,25.5,28.243969
16,MIA,Davion Mitchell,PG,NOP,31.737719,27.887386,20.5,21.764332
78,MIA,Bam Adebayo,C,NOP,32.584702,30.225278,30.5,29.682154
19,MIA,Norman Powell,SG,NOP,32.088047,30.881419,31.5,30.832180
54,MIA,Jaime Jaquez Jr.,SF,NOP,27.769880,29.828778,26.5,26.941755


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
44,BRK,Michael Porter Jr.,SF,DEN,31.644651,32.302234,38.5,37.664158
39,BRK,Terance Mann,SG,DEN,26.127554,26.598918,13.5,14.117644


../tables/2025/gmday_preds_PRA.csv saved!
